In [1]:
file_path = '/Users/lfinkbeiner/Documents/GitHub/HERA/'
beam_origin = 'HERA_4.9m_E-pattern_151MHz.txt'
beam_destination = 'ant1_s2'

In [2]:
from RIMEz import beam_models
import numpy as np
from spin1_beam_model import cst_processing, jones_matrix_field

In [3]:
processor = cst_processing.CSTDataProcessor(
    [file_path + beam_origin,
    file_path + beam_origin,
    file_path + beam_origin],
    np.array([150e6, 151e6, 152e6]),
    1, 1e-4
)

In [5]:
processor.compute_spin1_harmonics()
processor.write_model_data(file_path, beam_destination)
spline_beam_func = beam_models.model_data_to_spline_beam_func(file_path + beam_destination + '.h5', np.array([150e6, 151e6, 152e6]))

//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:18: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.pos1_Elm_samples = h5f["pos1_Elm"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:19: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.neg1_Elm_samples = h5f["neg1_Elm"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:20: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.freq_nodes = h5f["frequencies"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:21: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.L_model = h5f["spatial_bandlimit"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:22: H5pyDeprec

delta_nu_in is 1000000.0


In [6]:
S = .5 * np.array([
        [1, 1, 0, 0],
        [0, 0, 1, 1j],
        [0, 0, 1, -1j],
        [1, -1, 0, 0]
    ])
def A_matrix(freq, alt, az):
    J = spline_beam_func(freq, alt, az)[0]
    J_outer = np.kron(J, np.conj(J))
    return np.dot(S, np.dot(J_outer, np.linalg.inv(S)))

In [7]:
# Delete this
print(A_matrix(151e6, np.array([np.pi / 4]), np.array([np.pi])))

[[-7.20266508e-07-6.07156038e-07j  4.93615423e-04-1.19490666e-03j
   4.93549352e-04+3.46835635e-04j -1.54006715e-07+3.23065006e-07j]
 [ 4.93409897e-04+9.74778890e-04j -1.48917287e-08-1.61965669e-08j
   5.11763584e-08-2.72520853e-08j -4.94284940e-04+1.25821129e-04j]
 [ 4.93446138e-04-1.25493571e-04j -5.11353449e-08+2.72696836e-08j
   1.49385636e-08+1.62141652e-08j -4.94321181e-04-9.74538265e-04j]
 [ 1.54732031e-07-3.23047407e-07j -4.94181726e-04-3.45916487e-04j
  -4.94115655e-04+1.19584792e-03j  7.21079672e-07+6.07138440e-07j]]


In [24]:
# to-do:
    # 1. see if we need radians or degrees
    # 2. convert hra-dec to az, alt
    # 3. combine plots for multiple Stokes parameters
import matplotlib.pyplot as plt

# Eventually: alt
x = np.linspace(0, 2 * np.pi, 100)

# Eventually: az
y = np.linspace(0, 2 * np.pi, 100)

I2I = A_matrix(151e6, x, y)[0][0]

# Why do we still have complex values? How, then, was figure 1 produced?

scaled_z = (I2I - I2I.min()) / I2I.ptp()
colors = plt.cm.viridis(scaled_z)

plt.scatter(x, y, marker='+', c=colors, s=150, linewidths=4)
plt.show()

//anaconda3/envs/hera/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in cdouble_scalars
  from ipykernel import kernelapp as app


TypeError: Cannot cast array data from dtype('complex128') to dtype('int64') according to the rule 'safe'